In [12]:
import asyncio
from poke_env.player.player import Player
from poke_env.player.random_player import RandomPlayer
import time
import pandas as pd
from BattleNode import BattleNode
from maxDamage import MaxDamagePlayer

In [13]:
typeChart = pd.read_csv('TypeTableEdited.csv')
typeChart.rename(columns = {'Unnamed: 0' : 'AttackType'}, inplace = True)
typeChart = typeChart.set_index('AttackType')

# each row is an attacking type
# each column is the defending type
# print(typeChart) #uncomment to see table

In [14]:
def calcDamage(node):
	if node.player_turn:
		
        # Prepare some variables
		player_t1 = node.battle.active_pokemon.type_1
		player_t2 = ""
		player_m_t = node.move.type
		move_basepow = node.move.base_power
		opp_t1 = node.battle.opponent_active_pokemon.type_1
		opp_t2 = ""
		
		# Check if our pokemon has 2nd type
		if node.battle.active_pokemon.type_2 != None:
			player_t2 = node.battle.active_pokemon.type_2
		
		# Check if Opponent pokemon has 2nd type
		if node.battle.opponent_active_pokemon.type_2 != None:
			opp_t2 = node.battle.opponent_active_pokemon.type_2
		
		# First: check for stab
		stab_bonus = 1 #Default value, means no bonus

		if player_t2 != "":
			if player_t1 == player_m_t or player_t2 == player_m_t: #if type1 == move type or type 2 == move type, then stab bonus
				stab_bonus =  1.5
		else:
			if player_t1 == player_m_t: #if type1 == move type, then stab bonus
				stab_bonus =  1.5
		
		# Second: Get multiplier
		# Checks the DataFrame in pokeType for the multiplier for this move
		# get multi for type 1
		t1_multi = typeChart.loc[player_m_t.name, opp_t1.name]
		t2_multi = ""
		if opp_t2 != "" : #two types for opponent
			t2_multi = typeChart.loc[player_m_t.name, opp_t2.name]

		# Third: multiply stab, multis, and base power
		if t2_multi != "" :
			return  stab_bonus * t1_multi * t2_multi * move_basepow
		else:
			return stab_bonus * t1_multi * move_basepow


	else: #is opponent turn
		player_t1 = node.battle.active_pokemon.type_1
		player_t2 = ""
		opp_mt = node.move.type
		move_basepow = node.move.base_power
		opp_t1 = node.battle.opponent_active_pokemon.type_1
		opp_t2 = ""
		
		# Check if our pokemon has 2nd type
		if node.battle.active_pokemon.type_2 != None:
			player_t2 = node.battle.active_pokemon.type_2
		
		# Check if Opponent pokemon has 2nd type
		if node.battle.opponent_active_pokemon.type_2 != None:
			opp_t2 = node.battle.opponent_active_pokemon.type_2
		
		# First: check for stab
		stab_bonus = 1 #Default value, means no bonus

		if opp_t2 != "":
			if opp_t1 == opp_mt or opp_t2 == opp_mt: #if type1 == move type or type 2 == move type, then stab bonus
				stab_bonus =  1.5
		else:
			if opp_t1 == opp_mt: #if type1 == move type, then stab bonus
				stab_bonus =  1.5
		
		# Second: Get multiplier
		# Checks the DataFrame in pokeType for the multiplier for this move
		# get multi for type 1
		t1_multi = typeChart.loc[opp_mt.name, player_t1.name]
		t2_multi = ""
		if player_t2 != "" : #two types for opponent
			t2_multi = typeChart.loc[opp_mt.name, player_t2.name]

		# Third: multiply stab, multis, and base power
		if t2_multi != "" :
			return  stab_bonus * t1_multi * t2_multi * move_basepow
		else:
			return stab_bonus * t1_multi * move_basepow

In [15]:
def buildTree(node):
	'''	Builds a 2 or 3 level tree of possible moves for the current battle state.
		Returns root node which is the current battle state. '''
	if(node.level != None and node.level >= 3):
		if node.player_turn == False:
			raise Exception("Base node is opponent turn. Base nodes should only be for the player's turn.")
		return node #only want 3 layers, so stop making children
	else:
		if(node.player_turn == None or node.player_turn == False): #if first layer (none) or if parent is opponent turn, now it is player turn
			# Create array where status moves are at the end
			moves = []
			statMoves = []
			for m in node.battle.available_moves:
				if m.base_power != 0:
					moves.append(m)
				else:
					statMoves.append(m)
			moves = moves + statMoves

			# Add each available move as a node for the player's turn
			for i in range(0, len(moves)):
				child = BattleNode(node.battle, True, node)
				if node.level == None:
						child.level = 1
				else: child.level = node.level + 1
				child.move = (moves[i])

				# Calculate the possible damage for that move
				child.damage = calcDamage(child)

				# Get the state representation of the node
				estState(child)

				# go to next layer of branch
				buildTree(child)
				# when it comes back, add to parent
				node.addChild(child)

			# Check for if we should switch
			if len(node.battle.opponent_active_pokemon.moves.values()) > 0:
				
				# Check move type vs our 2 types to get the multipler 
				for move in node.battle.opponent_active_pokemon.moves.values():
					# Prepare some variables
					player_t1 = node.battle.active_pokemon.type_1
					player_t2 = ""
					opp_mt = move.type
					
					# Check if our pokemon has 2nd type
					if node.battle.active_pokemon.type_2 != None:
						player_t2 = node.battle.active_pokemon.type_2
						
					t1_multi = typeChart.loc[opp_mt.name, player_t1.name]
					t2_multi = 1
					if player_t2 != "" : #two types for opponent
						t2_multi = typeChart.loc[opp_mt.name, player_t2.name]					

					breakOut = False
					# If the multiper is > 2, build a switch node
					if t1_multi * t2_multi > 2:
						# print(node.battle.active_pokemon, "add switch since multi: ", t1_multi, t2_multi, node.battle.active_pokemon.types, move, move.type)
						

						potentialSwitches = {}
						
						if len(node.battle.available_switches) > 0:
							# Decide who to switch to by going through each available switch
							for poke in node.battle.available_switches:
								
								# Check type against opp_mt
								p_t1 = poke.type_1
								p_t2 = ""

								if poke.type_2 != None:
									p_t2 = poke.type_2

								t1_multi = typeChart.loc[opp_mt.name, p_t1.name]
								t2_multi = 1

								if p_t2 != "" : #two types for opponent
									t2_multi = typeChart.loc[opp_mt.name, p_t2.name]

								# If the multiper is 0, check HP
								if t1_multi * t2_multi == 0:
									if poke.current_hp_fraction > .50:
										# Pick to switch and break
										child = BattleNode(node.battle, True, node)
										if node.level == None:
												child.level = 1
										else: child.level = node.level + 1
										child.move = poke
										breakOut = True
										break

								elif t1_multi * t2_multi <= 1:
									# Save in dict of potential switches
									potentialSwitches[poke] = t1_multi * t2_multi

							if breakOut:
								# Get the state representation of the node
								estState(child)		
								# go to next layer of branch
								buildTree(child)
								# when it comes back, add to parent
								node.addChild(child)
								break
							else:
								if len(potentialSwitches) == 0:
									print("switches are empty, don't make a node", node.battle.available_switches, node.battle.opponent_active_pokemon.moves.values())
								else:
									child = BattleNode(node.battle, True, node)
									if node.level == None:
											child.level = 1
									else: child.level = node.level + 1
									# Look over the list of switches and pick min
									minMulti = 999
									for p in potentialSwitches.keys():
										if potentialSwitches.get(p) < minMulti:
											minMulti = potentialSwitches.get(p)
											child.move = p

									# Get the state representation of the node
									estState(child)		
									# go to next layer of branch
									buildTree(child)
									# when it comes back, add to parent
									node.addChild(child)
			
		else:

			# If it is the opponent's turn, then nodes are a few of their possible moves
			if len(node.battle.opponent_active_pokemon.moves) < 2:
				# If we do not know any possible moves yet, or only 1, just return to the parent and don't make another layer
				return
				
			else:
				# create a node for each of the known moves
				for move in node.battle.opponent_active_pokemon.moves.values(): 
					child = BattleNode(node.battle, False, node)
					if node.level == None:
						child.level = 1
					else: child.level = node.level + 1
					child.move = move 

					# Calculate the possible damage for that move
					child.damage = calcDamage(child)

					# Get the state representation of the node
					estState(child)
					
					# go to next layer of branch
					buildTree(child)
					
					# when it comes back, add to parent
					node.addChild(child)

		# return node, should be the root when all the recursive calls come back
		return node

In [16]:
def showNodeTree(root):
	# Use the version below for all information for each node
	# print("Root: ", root.id, root.player_turn)
	# i = 0
	# for child in root.children:
	# 	print("\tPlayer Node ", i, " : ", child)
	# 	if len(child.children) > 0:
	# 		j = 0
	# 		for c in child.children:
	# 			print("\t\t Opp Node ", j, " : ", c)
	# 			if len(c.children) > 0:
	# 				k = 0
	# 				for kid in c.children:
	# 					print("\t\t\tplayer node ", k, " : ", kid)
	# 					k+=1
	# 			j+=1
	# 	i+=1

	#Use the one below for minimal information for each node
	print("Root: ", root.move, root.value)
	i = 0
	for child in root.children:
		print("\tPlayer Node ", i, " : ", child.move, "damage:", child.damage, "value:",  child.value, "\n\t\tState:", child.state)
		if len(child.children) > 0:
			j = 0
			for c in child.children:
				print("\t\t Opp Node ", j, " : ", c.move, "damage:", c.damage, "value:",  c.value, "\n\t\tState:", child.state)
				if len(c.children) > 0:
					k = 0
					for kid in c.children:
						print("\t\t\tplayer node ", k, " : ", kid.move, "damage:", kid.damage, "value:",  kid.value,  "\n\t\tState:", child.state)
						k+=1
				j+=1
		i+=1

In [17]:

def estState(node):
    # Take possible damage from each parent, and estimate the state of our hp and the opponent hp
    # Go from child to parent and then back down, changing the values on the way down
    # Assign the value to the node as the game state value which is a dictonary of
        # est player hp: value
        # est opp hp: value
    damageDivisor = 100.0

    # Check for if the node is a switch
    if node.damage == None:
        
        node.state = {
            "PlayerHP"  :  node.battle.active_pokemon.current_hp,
            "OppHP"     :  node.battle.opponent_active_pokemon.current_hp_fraction,
            "PlayerTot" :  node.battle.active_pokemon.current_hp, 
            "OppOrig"   :  node.battle.opponent_active_pokemon.current_hp_fraction
        }
        return

    if node.level == 1: #if it is the first level, then get the hp values from the source
        opp_hp_fraction = node.battle.opponent_active_pokemon.current_hp_fraction
        player_damage_fraction = node.damage / damageDivisor

        player_hp = node.battle.active_pokemon.current_hp
        opponent_damage = node.damage

    elif node.level > 1: #if it is after the first level, then use the previous node's state
        opp_hp_fraction = node.parent.state.get("OppHP")
        player_damage_fraction = node.damage / damageDivisor

        player_hp = node.parent.state.get("PlayerHP")
        opponent_damage = node.damage

    playerTotal = node.battle.active_pokemon.current_hp
    oppHpOrig = node.battle.opponent_active_pokemon.current_hp_fraction

    if node.player_turn:
        # Contains player move: calc affect on opponent HP
        node.state = {
            "PlayerHP"  :  player_hp,
            "OppHP"     :  opp_hp_fraction - player_damage_fraction,
            "PlayerTot" :  playerTotal, 
            "OppOrig"   :  oppHpOrig
        }
    
    else: 
        # Contains opponent move: calc affect on player HP
        node.state = {
            "PlayerHP"  :  player_hp - opponent_damage,
            "OppHP"     :  opp_hp_fraction,
            "PlayerTot" :  playerTotal, 
            "OppOrig"   :  oppHpOrig
        }
    

In [18]:
def minmax(node):
	'''	Goes through the battle tree recursively to evaluate the nodes.
		Then chooses a best move and sends that node back to choose_move.'''

	if len(node.children) < 1:
		# This is the last node of a branch, needs a value assigned
		# Based on the state, decide the value:
			# high value on opponent HP dropping and our Hp remaning
			# value lowers as opponent HP is higher, and if our HP lowers

		# Check for if the node is a switch, if switch is an option, we want to place a very high value on it
		if node.damage == None:
			node.value = 30
			return

		# Otherwise evaluate the move node
		playerHP = node.state.get("PlayerHP")
		oppHP = node.state.get("OppHP")
		playerTotal = node.battle.active_pokemon.current_hp
		oppHpOrig = node.battle.opponent_active_pokemon.current_hp_fraction

		# Calc value for oppHP

		if oppHP < -.60: 
			node.value = 20

		elif oppHP >= -.60 and oppHP < -.40 :
			node.value = 18

		elif oppHP >= -.40 and oppHP < -.30 :
			node.value = 16

		elif oppHP >= -.30 and oppHP < -.20 :
			node.value = 14

		elif oppHP >= -.20 and oppHP < -.10 :
			node.value = 12

		elif oppHP >= -.10 and oppHP < 0 :
			node.value = 10

		elif oppHP >= 0 and oppHP < .10 :
			node.value = 8

		elif oppHP >= .10 and oppHP < .20 :
			node.value = 6

		elif oppHP >= .20 and oppHP < .30 :
			node.value = 4

		elif oppHP >= .30 and oppHP < .40 :
			node.value = 2

		elif oppHP >= .40 and oppHP < .50 :
			node.value = 0

		elif oppHP >= .50 and oppHP < .60 :
			node.value = -2

		elif oppHP >= .60 and oppHP < .70 :
			node.value = -4

		elif oppHP >= .70 and oppHP < .80 :
			node.value = -6

		elif oppHP >= .80:
			node.value = -8
		
		else: #should not happen
			node.value = 0
			
		# Calc value for PlayerHP
		if playerHP >= .75*playerTotal:
			node.value += 5

		elif (playerHP > .25*playerTotal and playerHP < .80*playerTotal):  
			node.value += 0

		elif playerHP <= .25*playerTotal:
			node.value += -5

		else: #should not happen
			node.value = 0
		
		return
			
	else:

		# go to end of each branch recursively
		for i in range(0, len(node.children)):
			minmax(node.children[i])

		# Comes back when all children have a value
		# Get value for node from children based on turn
		if node.player_turn: 
			minVal = 9999 #picks min of opponent moves
			for child in node.children:
				if child.value < minVal:
					minVal = child.value
			node.value = minVal 
			
		else: #if opponent turn
			maxVal = -9999 #picks max of player moves
			for child in node.children:
				if child.value > maxVal:
					maxVal = child.value
			node.value = maxVal 
	
	# if the node is the root (no parent)
	if node.parent == None:
		
		# find best child and return their move
		if node.player_turn == None: # root should always be player turn
			# Pick the highest of player moves for chosen move for root
			best_move = None
			maxVal = -9999
			switch = ""
			for child in node.children:
				if child.value > maxVal:
					if child.damage == None: switch = child.move
					maxVal = child.value
					best_move = child.move
			node.move = best_move
			node.value = maxVal

			# if switch != "": print("chose to switch to ", switch, maxVal) #Uncomment to easier find a case where switch was chosen
			
			return best_move

		else: 
			print("Error: root not player turn")
			return 

In [19]:
class MinMaxPlayer(Player):
	
	def choose_move(self, battle):
		if battle.available_moves:
			
			rootNode = BattleNode(battle, None, None) #Create root node to start

			rootNode = buildTree(rootNode) #send root, returns root node of constructed tree
			
			best_move = minmax(rootNode) # send in root node(current battle), get chosen move
			
			if battle.can_dynamax: #if dynamax is available, use it
				return self.create_order(best_move, dynamax=True)

			# showNodeTree(rootNode) #Uncomment this to get info about the tree for each choice
				
			return self.create_order(best_move)
		else: 
			return self.choose_random_move(battle)

In [24]:
#Create random Opponent
random_player = RandomPlayer(battle_format="gen8randombattle")

#Create maxDamage opponent
maxDamage_player = MaxDamagePlayer(battle_format="gen8randombattle")

#Create minMaxPlayer
mmP = MinMaxPlayer(battle_format="gen8randombattle")

numBattles = 50  #change for the number of battles to run for each opponent

start = time.time()
# run for set number of games
await mmP.battle_against(random_player, n_battles=numBattles)

# print results 
print(
	"MinMax player won %d / %d battles against random player [this took %f seconds]"
	% (
		mmP.n_won_battles, numBattles, time.time() - start
	)
)

print("\n\n\n\n\n")

mmP2 = MinMaxPlayer(battle_format="gen8randombattle")
start = time.time()
await mmP2.battle_against(maxDamage_player, n_battles=numBattles)

# print results 
print(
	"MinMax player won %d / %d battles against maxDamage player [this took %f seconds]"
	% (
		mmP2.n_won_battles, numBattles, time.time() - start
	)
)

MinMax player won 49 / 50 battles against random player [this took 10.035847 seconds]






MinMax player won 44 / 50 battles against maxDamage player [this took 7.021055 seconds]
